## FastText Embedding Model 생성

### [크롤링 데이터 병합]

In [25]:
import os
import pandas as pd
import numpy as np
Youtube_path = "./Youtube_comments.csv"
ilbe_path = "./ilbe_comments.csv"

Comment_df = pd.DataFrame()

df1 = pd.read_csv(Youtube_path, encoding="cp949", header=None, names=[0])
df2 = pd.read_csv(ilbe_path, encoding="cp949", header=None, names=[0])
            
Comment_df = pd.concat([Comment_df, df1])
Comment_df = pd.concat([Comment_df, df2])
        
print(Youtube_path, len(df1), len(Comment_df))
print(ilbe_path, len(df2), len(Comment_df))

# dataset.txt, fword_list.txt 넣기
with open('dataset.txt', 'r', encoding='UTF8') as f:
    dataset_list = []
    i = 0
    while True:
        line = f.readline()
        if line == '' :
            break
        dataset_list.append(line.strip().split('|')[0])
        i += 1
    df3 = pd.Series(dataset_list)
    Comment_df = pd.concat([Comment_df, df3])
    print("dataset.txt", i, len(Comment_df))

with open('fword_list.txt', 'r', encoding='UTF8') as f:
    dataset_list = []
    i = 0
    while True:
        line = f.readline()
        if line == '' :
            break
        dataset_list.append(line)
        i += 1
    df4 = pd.Series(dataset_list)
    Comment_df = pd.concat([Comment_df, df4])
    print("fword_list.txt", i, len(Comment_df))

./Youtube_comments.csv 91281 228250
./ilbe_comments.csv 136969 228250
dataset.txt 5825 234075
fword_list.txt 3577 237652


### [FastText input]

In [26]:
# 중복제거 결과: 237652 -> 219241
Comment_df.drop_duplicates(inplace=True)
print(len(Comment_df))

219241


In [27]:
import re
pattern = re.compile("[^ㄱ-ㅎㅏ-ㅣ가-힣0-9a-zA-Z ]") # 특수문자 제거

# 각 단어에 정규표현식 적용
def clear_word(word):
    word = re.sub(pattern, "", word)
    return word

Comment_df[0] = Comment_df.astype('str')
Comment_df[0] = Comment_df[0].apply(lambda x: clear_word(x))

In [34]:
CHOSUNG_LIST = [u'ㄱ',u'ㄲ',u'ㄴ',u'ㄷ',u'ㄸ',u'ㄹ',u'ㅁ',u'ㅂ',u'ㅃ',u'ㅅ',u'ㅆ',u'ㅇ',u'ㅈ',u'ㅉ',u'ㅊ',u'ㅋ',u'ㅌ',u'ㅍ',u'ㅎ']
JOONGSUNG_LISTS = [u'ㅏ',u'ㅐ',u'ㅑ',u'ㅒ',u'ㅓ',u'ㅔ',u'ㅕ',u'ㅖ',u'ㅗ',u'ㅘ',u'ㅙ',u'ㅚ',u'ㅛ',u'ㅜ',u'ㅝ',u'ㅞ',u'ㅟ',u'ㅠ',u'ㅡ',u'ㅢ',u'ㅣ']
JONGSUNG_LIST = [u'_',u'ㄱ',u'ㄲ',u'ㄳ',u'ㄴ',u'ㄵ',u'ㄶ',u'ㄷ',u'ㄹ',u'ㄺ',u'ㄻ',u'ㄼ',u'ㄽ',u'ㄾ',u'ㄿ',u'ㅀ',u'ㅁ',u'ㅂ',u'ㅄ',u'ㅅ',u'ㅆ',u'ㅇ',u'ㅈ',u'ㅊ',u'ㅋ',u'ㅌ',u'ㅍ',u'ㅎ']

# 자모분리기
def kor_decompose(word, end_char="_"):
    result = []
    
    for char in word:
        char_unicode = ord(char)
        
        if 0xD7A3 < char_unicode or char_unicode < 0xAC00:
            result.append(char)
            continue

        chosung_index = int((((char_unicode - 0xAC00) / 28) / 21) % 19)
        joongsung_index = int(((char_unicode - 0xAC00) / 28) % 21)
        jongsung_index = int((char_unicode - 0xAC00) % 28)
        
        chosung = CHOSUNG_LIST[chosung_index]
        joongsung = JOONGSUNG_LISTS[joongsung_index]
        jongsung = JONGSUNG_LIST[jongsung_index]
        
        # 종성이 없을 경우 end_char
        if jongsung_index == 0:
            jongsung = end_char
        
        result.append(chosung)
        result.append(joongsung)
        result.append(jongsung)

    return "".join(result)

In [32]:
Comment_df.reset_index(inplace=True)
Comment_df.drop('index', axis=1, inplace=True)
Comment_df = Comment_df[1:]

In [36]:
# 자모분리 후 저장
Comment_df[0] = Comment_df[0].apply(lambda x: kor_decompose(x))
Comment_df[0] = Comment_df[0].apply(lambda x: x.split(" "))

In [37]:
Comment_df

,0
1,"[ㅁㅣ_ㅇㅕㄴㅇㅣ_, ㄷㅔ_ㅂㅟ_ㅎㅏ_ㅁㅕㄴ, ㅁㅣ_ㅁㅗ_ㄹㅗ_, ㄷㅏ_, ㅆㅣㅂㅇ..."
2,"[ㅋㅠ_ㅂㅡ_ㅇㅑ_, ㅇㅏ_ㅇㅣ_ㄷㅡㄹ, ㅇㅖ_ㄴㅡㅇ, ㅈㅗㅁ, ㅁㅏㄶㅇㅣ_, ㄴㅐ..."
3,"[ㅋㅓㅁㅂㅐㄱㅎㅏㄹ, ㄸㅐ_, ㅁㅏ_ㄷㅏ_, ㅍㅏ_ㅌㅡ_ㅂㅜㄴㅂㅐ_ㄹㅡㄹ, ㅈㅏㄹ,..."
4,"[ㅈㅣㄴㅅㅣㅁ, ㅇㅏ_ㅇㅣ_ㄷㅗㄹ, ㄴㅏ_ㅇㅗ_ㅁㅕㄴ, ㅎㅏㄴㄷㅜ_ㅁㅕㅇㅇㅡㄴ, ㅈ..."
5,"[ㅇㅘ_, ㅈㅗㄴㄴㅏ_, ㅇㅖ_ㅃㅡ_ㄷㅏ_, ㅇㅝㄴㄹㅐ_ㄷㅗ_, ㅇㅖ_ㅃㅓㅆㅈㅣ_ㅁ..."
...,...
219235,"[ㅇ, ㅐㅈㅏ_]"
219236,[ㅆㅣ_ㅂㅜㄹㅇㅏㄹ]
219237,[ㅈㅗㄴㅁㅏ_ㄴㅣ_]
219238,[ㄱㅣ_ㅈㅣㅂㄴㅕㄴ]


In [38]:
# list로 변환
Comment_list = list(Comment_df[0])
len(Comment_list)

219239

In [39]:
# fasttext input
Comment_list[:10]

[['ㅁㅣ_ㅇㅕㄴㅇㅣ_',
  'ㄷㅔ_ㅂㅟ_ㅎㅏ_ㅁㅕㄴ',
  'ㅁㅣ_ㅁㅗ_ㄹㅗ_',
  'ㄷㅏ_',
  'ㅆㅣㅂㅇㅓ_ㅁㅓㄱㅇㅡㄹㅈㅜㄹ',
  'ㅇㅏㄹㅇㅏㅆㄷㅏ_ㄴㅡㄴㄱㅓ_',
  'ㅇㅙ_ㅋㅔ_',
  'ㅇㅜㅅㄱㅕ_ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ'],
 ['ㅋㅠ_ㅂㅡ_ㅇㅑ_',
  'ㅇㅏ_ㅇㅣ_ㄷㅡㄹ',
  'ㅇㅖ_ㄴㅡㅇ',
  'ㅈㅗㅁ',
  'ㅁㅏㄶㅇㅣ_',
  'ㄴㅐ_ㅂㅗ_ㄴㅐ_ㅈㅝ_ㄹㅏ_',
  'ㅇㅗ_ㄴㅡㄹ',
  'ㅇㅏ_ㅎㅕㅇ',
  'ㄴㅓ_ㅁㅜ_',
  'ㅈㅐ_ㅁㅣㅆㅇㅓㅆㄷㅏ_',
  'ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ'],
 ['ㅋㅓㅁㅂㅐㄱㅎㅏㄹ',
  'ㄸㅐ_',
  'ㅁㅏ_ㄷㅏ_',
  'ㅍㅏ_ㅌㅡ_ㅂㅜㄴㅂㅐ_ㄹㅡㄹ',
  'ㅈㅏㄹ',
  'ㅁㅗㅅㅎㅏㄴㄷㅏ_ㄱㅗ_',
  'ㅁㅝ_ㄹㅏ_ㅎㅏ_ㄴㅡㄴㄷㅔ_',
  'ㄱㅡ_ㄹㅜㅂㅇㅔ_',
  'ㅈㅓㄴㅅㅗ_ㅇㅕㄴㅇㅣ_',
  'ㅇㅣㅆㄴㅡㄴ',
  'ㄱㅓㅅㅁㅏㄴㅇㅡ_ㄹㅗ_ㄷㅗ_',
  '',
  'ㅋㅡㄴ',
  'ㅎㅐㅇㅇㅜㄴㅇㅣㅁ',
  'ㄷㅐ_ㅊㅔ_',
  'ㅇㅓ_ㄸㅓㄴ',
  'ㄱㅡ_ㄹㅜㅂㅇㅔ_',
  'ㄹㅣ_ㄷㅓ_ㄱㅏ_',
  'ㅍㅡ_ㄹㅗ_ㄷㅠ_ㅅㅣㅇ',
  'ㅃㅜㄴㅁㅏㄴ',
  'ㅇㅏ_ㄴㅣ_ㄹㅏ_',
  'ㅁㅔㅁㅂㅓ_ㄷㅡㄹ',
  'ㅇㅡㅁㅅㅐㄱ',
  'ㅂㅏㄹㅇㅡㅁㄲㅏ_ㅈㅣ_',
  'ㅅㅣㄴㄱㅕㅇㅆㅓ_ㅅㅓ_',
  'ㄴㅗ_ㄹㅐ_ㄹㅡㄹ',
  'ㅁㅏㄴㄷㅡㄹㅇㅓ_ㅈㅜ_ㄱㅗ_',
  'ㅋㅓㅁㅂㅐㄱㅎㅏㄹㄸㅐ_',
  'ㅁㅏ_ㄷㅏ_',
  'ㅋㅓㄴㅅㅔㅂㅇㅡㄴ',
  'ㅁㅐ_ㅂㅓㄴ',
  'ㄷㅏ_ㄹㅡㄴㄷㅔ_',
  'ㄸㅗ_',
  'ㅁㅔㅁㅂㅓ_ㄷㅡㄹㅇㅣ_ㄹㅏㅇ',
  'ㅇㅓ_ㅇㅜㄹㄹㅣ_ㄱㅔ_',
  'ㅁㅏㄴㄷㅡㄹㅇㅓ_ㅈㅜ_ㄴㅑ_ㄱㅗ_',
  'ㅈㅓㄴㅅㅗ_ㅇㅕㄴㅇㅡㄴ',
  'ㅈㅣㄴㅉㅏ_',
  'ㅊㅚ_ㄱㅗ_ㅇㅢ_',
  'ㄹㅣ_ㄷㅓ_ㄷㅏ_',
  'ㄱㅡ_ㄹㅣ_ㄱㅗ_',
  'ㅈㅓㄴㅅㅗ_ㅇㅕㄴㅇㅢ_',
  'ㅅㅡ_ㅋㅔ_ㅊㅣ_ㄹㅡㄹ',
  'ㅇㅣㅆㄴㅡㄴ',
  'ㄱㅡ_ㄷㅐ_ㄹㅗ_',
  'ㅍㅛ_ㅎㅕㄴㅎㅐ_ㅈㅜ_ㄱㅗ_',
  'ㄷㅟ